# Driver Splits

Script to rebase the split times for a stage and display them relative to a specified driver.

The intention is to generate a report on a stage that is meaningful to a specified driver.

Ideally the report should:

- show where the driver finished on the stage (stage rank)
- show the running stage delta at each split compared to each other driver
- show the extent to which a driver gained or lost time on each split compared to each other driver
- show the start order (so that this can be related to stage rank)
- identify the overall position at the end of the stage for each driver
- show whether overall positions were gained or lost after the stage (not implemented yet; need a +=- column)

In [8]:
import notebookimport

if __name__=='__main__':
    rebase='overallleader'
    rebase='PAD'#'PAD'
    MAXINSPLITDELTA=20 #set xlim on the within split delta
    ss='SS19'
    drivercode=rebase

In [9]:
sr = __import__("Charts - Stage Results")
ssd = __import__("Charts - Split Sector Delta")

importing Jupyter notebook from Charts - Stage Results.ipynb
importing Jupyter notebook from Charts - Split Sector Delta.ipynb


In [10]:
import os
import sqlite3
import pandas as pd
import pytablewriter
import six

#dbname='wrc18.db'
#dbname='france18.db'
#conn = sqlite3.connect(dbname)

if __name__=='__main__':
    #dbname='wrc18.db'
    dbname='italy18.db'
    conn = sqlite3.connect(dbname)
    rally='Italy'
    rc='RC1'
    year=2018
    #ss='SS4'

In [11]:
stageresults = sr.dbGetRallyStages(conn, rally).sort_values('number')
stageresults.head()

,legDate,date,startListId,status,itineraryLegId,itinerarySectionId,section,order,code,distance,eventId,name,number,stageId,stageType,status,timingPrecision,itineraryLegId,itinerarySections.itinerarySectionId
0,2018-06-07,Thursday 7th June,153,Completed,92,198,Section 1,1,SS1,2.00,32,ITTIRI ARENA SHOW,1,500,HeadToHeadSuperSpecialStage,Completed,Tenth,92,198
1,2018-06-08,Friday 8th June,157,Completed,91,199,Section 2,2,SS2,22.12,32,TULA 1,2,502,SpecialStage,Completed,Tenth,91,199
2,2018-06-08,Friday 8th June,157,Completed,91,199,Section 2,2,SS3,14.37,32,CASTELSARDO 1,3,503,SpecialStage,Completed,Tenth,91,199
3,2018-06-08,Friday 8th June,157,Completed,91,199,Section 2,2,SS4,14.14,32,TERGU - OSILO 1,4,505,SpecialStage,Completed,Tenth,91,199
4,2018-06-08,Friday 8th June,157,Completed,91,199,Section 2,2,SS5,11.46,32,MONTE BARANTA 1,5,506,SpecialStage,Completed,Tenth,91,199


In [15]:
def rebaseOverallRallyTime(stagerank_overall, drivercode, code):
    rebase = stagerank_overall[stagerank_overall['driver.code']==drivercode][['code','totalTimeMs']].set_index('code').to_dict(orient='dict')['totalTimeMs']
    stagerank_overall['rebased'] = stagerank_overall['totalTimeMs'] - stagerank_overall['code'].map(rebase)
    return stagerank_overall

In [18]:
stagerank_overall = sr.getEnrichedStageRank(conn, rally, typ='overall')
stagerank_overall[stagerank_overall['code']==ss][['driver.code','position','totalTimeMs','code']]

,driver.code,position,totalTimeMs,code
263,OGI,1,12265000,SS19
264,NEU,2,12265800,SS19
265,LAP,3,12377100,SS19
266,PAD,4,12430600,SS19
267,OST,5,12445900,SS19
268,BRE,6,12529400,SS19
269,LAT,7,12942300,SS19
270,TÄN,8,13022400,SS19
271,PRO,10,13186300,SS19
272,SUN,11,13189300,SS19


In [19]:
zz=rebaseOverallRallyTime(stagerank_overall, 'PAD', ss)
zz=zz[zz['code']==ss][['driver.code','position','totalTimeMs','code', 'rebased']].set_index('driver.code')
zz['Overall Time']=-zz['rebased']/1000
zz

,position,totalTimeMs,code,rebased,Overall Time
driver.code,,,,,
OGI,1,12265000,SS19,-165600,165.6
NEU,2,12265800,SS19,-164800,164.8
LAP,3,12377100,SS19,-53500,53.5
PAD,4,12430600,SS19,0,0.0
OST,5,12445900,SS19,15300,-15.3
BRE,6,12529400,SS19,98800,-98.8
LAT,7,12942300,SS19,511700,-511.7
TÄN,8,13022400,SS19,591800,-591.8
PRO,10,13186300,SS19,755700,-755.7


In [20]:
stagerank_overall.columns

Index(['diffFirst', 'diffFirstMs', 'diffPrev', 'diffPrevMs', 'entryId',
       'penaltyTime', 'penaltyTimeMs', 'position', 'stageTime', 'stageTimeMs',
       'totalTime', 'totalTimeMs', 'stageId', 'class', 'code', 'distance',
       'name', 'snum', 'driver.code', 'entrant.name', 'classrank',
       'gainedClassPos', 'gainedClassLead', 'classPosDiff', 'lostClassLead',
       'retainedClassLead', 'gainedOverallPos', 'gainedOverallLead',
       'overallPosDiff', 'lostOverallLead', 'retainedOverallLead', 'stagewin',
       'stagewincount', 'winsinarow', 'gainedTime', 'rebased'],
      dtype='object')

In [21]:
stagerank_stage = sr.getEnrichedStageRank(conn, rally, typ='stage')
stagerank_stage[stagerank_stage['driver.code']=='PAD'][['position','code']]

,position,code
4,5,SS1
24,10,SS2
35,6,SS3
53,9,SS4
68,9,SS5
79,5,SS6
99,10,SS7
112,8,SS8
125,6,SS9
143,9,SS10


In [22]:
splits = ssd.dbGetSplits(conn,rally,ss,rc)

elapseddurations=ssd.getElapsedDurations(splits)
elapseddurations.head()

,drivercode,elapsedDurationS,startDateTime,section
0,EVA,97.0,2018-06-10T09:15:00,1
1,EVA,207.1,2018-06-10T09:15:00,2
2,EVA,290.1,2018-06-10T09:15:00,3
3,EVA,354.0,2018-06-10T09:15:00,4
4,EVA,442.9,2018-06-10T09:15:00,5


In [23]:
roadPos=splits[['drivercode','startDateTime']].drop_duplicates()
roadPos = roadPos.set_index('drivercode')
roadPos['Road Position']=roadPos['startDateTime'].rank().astype(int).astype(str)
roadPos

,startDateTime,Road Position
drivercode,,
EVA,2018-06-10T09:15:00,1
BRE,2018-06-10T09:18:00,2
OST,2018-06-10T09:21:00,3
PAD,2018-06-10T09:24:00,4
LAP,2018-06-10T09:27:00,5
LAT,2018-06-10T09:30:00,6
NEU,2018-06-10T09:33:00,7
OGI,2018-06-10T09:36:00,8
TÄN,2018-06-10T09:39:00,9


In [24]:
splits[splits['drivercode'].isin(['PAD','NEU'])]

,elapsedDuration,elapsedDurationMs,entryId,splitDateTime,splitDateTimeLocal,splitPointId,splitPointTimeId,stageTimeDuration,stageTimeDurationMs,startDateTime,startDateTimeLocal,stageId,class,code,distance,name,drivercode,elapsedDurationS
15,PT1M32.88S,92880,2280,2018-06-10T09:25:32.88Z,2018-06-10T11:25:32.88+02:00,1284,35841,00:08:47,527000.0,2018-06-10T09:24:00,2018-06-10T11:24:00+02:00,504,RC1,SS19,14.06,CALA FLUMINI 2,PAD,92.9
16,PT3M17.592S,197592,2280,2018-06-10T09:27:17.592Z,2018-06-10T11:27:17.592+02:00,1286,35844,00:08:47,527000.0,2018-06-10T09:24:00,2018-06-10T11:24:00+02:00,504,RC1,SS19,14.06,CALA FLUMINI 2,PAD,197.6
17,PT4M35.907S,275907,2280,2018-06-10T09:28:35.907Z,2018-06-10T11:28:35.907+02:00,1287,35846,00:08:47,527000.0,2018-06-10T09:24:00,2018-06-10T11:24:00+02:00,504,RC1,SS19,14.06,CALA FLUMINI 2,PAD,275.9
18,PT5M39.144S,339144,2280,2018-06-10T09:29:39.144Z,2018-06-10T11:29:39.144+02:00,1288,35848,00:08:47,527000.0,2018-06-10T09:24:00,2018-06-10T11:24:00+02:00,504,RC1,SS19,14.06,CALA FLUMINI 2,PAD,339.1
19,PT7M4.236S,424236,2280,2018-06-10T09:31:04.236Z,2018-06-10T11:31:04.236+02:00,1289,35851,00:08:47,527000.0,2018-06-10T09:24:00,2018-06-10T11:24:00+02:00,504,RC1,SS19,14.06,CALA FLUMINI 2,PAD,424.2
30,PT1M30.664S,90664,2279,2018-06-10T09:34:30.664Z,2018-06-10T11:34:30.664+02:00,1284,35857,00:08:28.9000000,508900.0,2018-06-10T09:33:00,2018-06-10T11:33:00+02:00,504,RC1,SS19,14.06,CALA FLUMINI 2,NEU,90.7
31,PT3M12.077S,192077,2279,2018-06-10T09:36:12.077Z,2018-06-10T11:36:12.077+02:00,1286,35860,00:08:28.9000000,508900.0,2018-06-10T09:33:00,2018-06-10T11:33:00+02:00,504,RC1,SS19,14.06,CALA FLUMINI 2,NEU,192.1
32,PT4M26.496S,266496,2279,2018-06-10T09:37:26.496Z,2018-06-10T11:37:26.496+02:00,1287,35862,00:08:28.9000000,508900.0,2018-06-10T09:33:00,2018-06-10T11:33:00+02:00,504,RC1,SS19,14.06,CALA FLUMINI 2,NEU,266.5
33,PT5M28.369S,328369,2279,2018-06-10T09:38:28.369Z,2018-06-10T11:38:28.369+02:00,1288,35863,00:08:28.9000000,508900.0,2018-06-10T09:33:00,2018-06-10T11:33:00+02:00,504,RC1,SS19,14.06,CALA FLUMINI 2,NEU,328.4
34,PT6M50.01S,410010,2279,2018-06-10T09:39:50.01Z,2018-06-10T11:39:50.01+02:00,1289,35865,00:08:28.9000000,508900.0,2018-06-10T09:33:00,2018-06-10T11:33:00+02:00,504,RC1,SS19,14.06,CALA FLUMINI 2,NEU,410.0


In [25]:
rebasedelapseddurations = ssd.rebaseElapsedDurations(elapseddurations, drivercode)
rebasedelapseddurations.head()

,drivercode,elapsedDurationS,startDateTime,section,rebased
0,EVA,97.0,2018-06-10T09:15:00,1,4.1
1,EVA,207.1,2018-06-10T09:15:00,2,9.5
2,EVA,290.1,2018-06-10T09:15:00,3,14.2
3,EVA,354.0,2018-06-10T09:15:00,4,14.9
4,EVA,442.9,2018-06-10T09:15:00,5,18.7


In [26]:
rbe=-rebasedelapseddurations.pivot('drivercode','section','rebased')
rbe.columns=list(rbe.columns)[:-1]+['{} Overall'.format(ss)]
rbe=rbe.sort_values(rbe.columns[-1],ascending =False)
rbe

,1,2,3,4,5,SS19 Overall
drivercode,,,,,,
NEU,2.2,5.5,9.4,10.7,14.2,18.1
OGI,2.7,5.6,9.5,10.4,13.5,17.6
TÄN,2.2,5.0,8.0,8.2,11.1,14.1
LAT,0.5,2.8,4.8,5.8,7.6,9.8
SUN,0.8,0.7,1.3,1.8,2.0,2.0
PAD,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
LAP,-0.7,-1.1,-1.2,-0.5,-0.4,-0.3
BRE,-0.0,-1.2,-1.0,-1.6,-2.0,-3.0
OST,-0.6,-2.1,-3.2,-2.8,-3.6,-4.1


In [27]:
#https://pandas.pydata.org/pandas-docs/stable/style.html
def color_negative(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if isinstance(val, str): return ''
    elif val and (isinstance(val,int) or isinstance(val,float)):
        color = 'green' if val < 0 else 'red' if val > 0  else 'black'
    else:
        color='white'
    return 'color: %s' % color

In [28]:
s = rbe.style.applymap(color_negative)
s

,1,2,3,4,5,SS19 Overall
drivercode,,,,,,
NEU,2.2,5.5,9.4,10.7,14.2,18.1
OGI,2.7,5.6,9.5,10.4,13.5,17.6
TÄN,2.2,5,8,8.2,11.1,14.1
LAT,0.5,2.8,4.8,5.8,7.6,9.8
SUN,0.8,0.7,1.3,1.8,2,2
PAD,-0,-0,-0,-0,-0,-0
LAP,-0.7,-1.1,-1.2,-0.5,-0.4,-0.3
BRE,-0,-1.2,-1,-1.6,-2,-3
OST,-0.6,-2.1,-3.2,-2.8,-3.6,-4.1


In [29]:
splitdurations = ssd.getSplitDurationsFromSplits(conn,rally,ss,rc)
splitdurations.head()

,drivercode,splitDurationS,startDateTime,stageTimeDurationMs,section
0,EVA,97.0,2018-06-10T09:15:00,551000.0,1
1,EVA,110.2,2018-06-10T09:15:00,551000.0,2
2,EVA,82.9,2018-06-10T09:15:00,551000.0,3
3,EVA,63.9,2018-06-10T09:15:00,551000.0,4
4,EVA,89.0,2018-06-10T09:15:00,551000.0,5


In [30]:
rebasedSplits = ssd.rebaseSplitDurations(splitdurations, drivercode)
rebasedSplits.head()

,drivercode,splitDurationS,startDateTime,stageTimeDurationMs,section,rebased
0,EVA,97.0,2018-06-10T09:15:00,551000.0,1,4.1
1,EVA,110.2,2018-06-10T09:15:00,551000.0,2,5.5
2,EVA,82.9,2018-06-10T09:15:00,551000.0,3,4.6
3,EVA,63.9,2018-06-10T09:15:00,551000.0,4,0.7
4,EVA,89.0,2018-06-10T09:15:00,551000.0,5,3.9


In [31]:
splitdurations[splitdurations['drivercode'].isin( ['PAD','NEU'])]

,drivercode,splitDurationS,startDateTime,stageTimeDurationMs,section,rebased
15,PAD,92.9,2018-06-10T09:24:00,527000.0,1,0.0
16,PAD,104.7,2018-06-10T09:24:00,527000.0,2,0.0
17,PAD,78.3,2018-06-10T09:24:00,527000.0,3,0.0
18,PAD,63.2,2018-06-10T09:24:00,527000.0,4,0.0
19,PAD,85.1,2018-06-10T09:24:00,527000.0,5,0.0
73,PAD,102.8,2018-06-10T09:24:00,527000.0,6,0.0
30,NEU,90.7,2018-06-10T09:33:00,508900.0,1,-2.2
31,NEU,101.4,2018-06-10T09:33:00,508900.0,2,-3.3
32,NEU,74.4,2018-06-10T09:33:00,508900.0,3,-3.9
33,NEU,61.9,2018-06-10T09:33:00,508900.0,4,-1.3


In [32]:
rbp=-rebasedSplits.pivot('drivercode','section','rebased')
rbp.columns=['D{}'.format(c) for c in rbp.columns]
rbp.sort_values(rbp.columns[-1],ascending =True)

,D1,D2,D3,D4,D5,D6
drivercode,,,,,,
PIA,-13.1,-15.7,-14.1,-12.4,-35.0,-44.6
MIK,-10.6,-10.1,-7.0,-6.4,-8.6,-10.3
EVA,-4.1,-5.5,-4.6,-0.7,-3.9,-5.3
PRO,-5.0,-5.8,-5.0,-3.4,-5.0,-3.3
BRE,-0.0,-1.1,0.1,-0.6,-0.4,-1.0
OST,-0.6,-1.5,-1.2,0.4,-0.8,-0.5
PAD,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
SUN,0.8,-0.1,0.6,0.5,0.2,-0.0
LAP,-0.7,-0.4,-0.1,0.7,0.1,0.1


In [33]:
roadPos

,startDateTime,Road Position
drivercode,,
EVA,2018-06-10T09:15:00,1
BRE,2018-06-10T09:18:00,2
OST,2018-06-10T09:21:00,3
PAD,2018-06-10T09:24:00,4
LAP,2018-06-10T09:27:00,5
LAT,2018-06-10T09:30:00,6
NEU,2018-06-10T09:33:00,7
OGI,2018-06-10T09:36:00,8
TÄN,2018-06-10T09:39:00,9


In [34]:
rb2=pd.merge(rbe,rbp,left_index=True, right_index=True)
rb2=pd.merge(rb2,zz[['position','Overall Time']],left_index=True, right_index=True)
rb2=pd.merge(rb2,roadPos[['Road Position']],left_index=True, right_index=True)
cols=rb2.columns.tolist()
rb2=rb2[[cols[-1]]+cols[:-1]]
rb2

,Road Position,1,2,3,4,5,SS19 Overall,D1,D2,D3,D4,D5,D6,position,Overall Time
NEU,7,2.2,5.5,9.4,10.7,14.2,18.1,2.2,3.3,3.9,1.3,3.5,3.9,2,164.8
OGI,8,2.7,5.6,9.5,10.4,13.5,17.6,2.7,2.9,3.9,0.9,3.1,4.1,1,165.6
TÄN,9,2.2,5.0,8.0,8.2,11.1,14.1,2.2,2.8,3.0,0.2,2.9,3.0,8,-591.8
LAT,6,0.5,2.8,4.8,5.8,7.6,9.8,0.5,2.3,2.1,1.0,1.7,2.2,7,-511.7
SUN,10,0.8,0.7,1.3,1.8,2.0,2.0,0.8,-0.1,0.6,0.5,0.2,-0.0,11,-758.7
PAD,4,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,4,0.0
LAP,5,-0.7,-1.1,-1.2,-0.5,-0.4,-0.3,-0.7,-0.4,-0.1,0.7,0.1,0.1,3,53.5
BRE,2,-0.0,-1.2,-1.0,-1.6,-2.0,-3.0,-0.0,-1.1,0.1,-0.6,-0.4,-1.0,6,-98.8
OST,3,-0.6,-2.1,-3.2,-2.8,-3.6,-4.1,-0.6,-1.5,-1.2,0.4,-0.8,-0.5,5,-15.3
EVA,1,-4.1,-9.5,-14.2,-14.9,-18.7,-24.0,-4.1,-5.5,-4.6,-0.7,-3.9,-5.3,14,-907.4


In [35]:
rb2.dtypes

Road Position     object
1                float64
2                float64
3                float64
4                float64
5                float64
SS19 Overall     float64
D1               float64
D2               float64
D3               float64
D4               float64
D5               float64
D6               float64
position           int64
Overall Time     float64
dtype: object

In [36]:
#There seems to be missing tenths?
#Elapsed durations are provided in milliseconds. Need to round correctly to tenths?
#Elapsed times grabbed from ssd.dbGetSplits(conn,rally,ss,rc)
from numpy import NaN
rb2=rb2.replace(0,NaN)
#rb2=rb2.fillna('') #This casts columns containing NA to object type
rb2['position']=rb2['position'].astype(str)
rb2.rename(columns={'position': 'Overall Position'}, inplace=True)

In [37]:
rb2.dtypes

Road Position        object
1                   float64
2                   float64
3                   float64
4                   float64
5                   float64
SS19 Overall        float64
D1                  float64
D2                  float64
D3                  float64
D4                  float64
D5                  float64
D6                  float64
Overall Position     object
Overall Time        float64
dtype: object

In [38]:
s=rb2.fillna('').style.applymap(color_negative)
#data.style.applymap(highlight_cols, subset=pd.IndexSlice[:, ['B', 'C']])

s.set_caption("{}: running split times and deltas within each split.".format(ss))

,Road Position,1,2,3,4,5,SS19 Overall,D1,D2,D3,D4,D5,D6,Overall Position,Overall Time
NEU,7,2.2,5.5,9.4,10.7,14.2,18.1,2.2,3.3,3.9,1.3,3.5,3.9,2,164.8
OGI,8,2.7,5.6,9.5,10.4,13.5,17.6,2.7,2.9,3.9,0.9,3.1,4.1,1,165.6
TÄN,9,2.2,5,8,8.2,11.1,14.1,2.2,2.8,3,0.2,2.9,3,8,-591.8
LAT,6,0.5,2.8,4.8,5.8,7.6,9.8,0.5,2.3,2.1,1,1.7,2.2,7,-511.7
SUN,10,0.8,0.7,1.3,1.8,2,2,0.8,-0.1,0.6,0.5,0.2,,11,-758.7
PAD,4,,,,,,,,,,,,,4,
LAP,5,-0.7,-1.1,-1.2,-0.5,-0.4,-0.3,-0.7,-0.4,-0.1,0.7,0.1,0.1,3,53.5
BRE,2,,-1.2,-1,-1.6,-2,-3,,-1.1,0.1,-0.6,-0.4,-1,6,-98.8
OST,3,-0.6,-2.1,-3.2,-2.8,-3.6,-4.1,-0.6,-1.5,-1.2,0.4,-0.8,-0.5,5,-15.3
EVA,1,-4.1,-9.5,-14.2,-14.9,-18.7,-24,-4.1,-5.5,-4.6,-0.7,-3.9,-5.3,14,-907.4


In [39]:
html=s.render()

In [40]:
from IPython.core.display import HTML
HTML(html)

,Road Position,1,2,3,4,5,SS19 Overall,D1,D2,D3,D4,D5,D6,Overall Position,Overall Time
NEU,7,2.2,5.5,9.4,10.7,14.2,18.1,2.2,3.3,3.9,1.3,3.5,3.9,2,164.8
OGI,8,2.7,5.6,9.5,10.4,13.5,17.6,2.7,2.9,3.9,0.9,3.1,4.1,1,165.6
TÄN,9,2.2,5,8,8.2,11.1,14.1,2.2,2.8,3,0.2,2.9,3,8,-591.8
LAT,6,0.5,2.8,4.8,5.8,7.6,9.8,0.5,2.3,2.1,1,1.7,2.2,7,-511.7
SUN,10,0.8,0.7,1.3,1.8,2,2,0.8,-0.1,0.6,0.5,0.2,,11,-758.7
PAD,4,,,,,,,,,,,,,4,
LAP,5,-0.7,-1.1,-1.2,-0.5,-0.4,-0.3,-0.7,-0.4,-0.1,0.7,0.1,0.1,3,53.5
BRE,2,,-1.2,-1,-1.6,-2,-3,,-1.1,0.1,-0.6,-0.4,-1,6,-98.8
OST,3,-0.6,-2.1,-3.2,-2.8,-3.6,-4.1,-0.6,-1.5,-1.2,0.4,-0.8,-0.5,5,-15.3
EVA,1,-4.1,-9.5,-14.2,-14.9,-18.7,-24,-4.1,-5.5,-4.6,-0.7,-3.9,-5.3,14,-907.4


In [41]:
#https://community.modeanalytics.com/gallery/python_dataframe_styling/
# Set CSS properties for th elements in dataframe
th_props = [
  ('font-size', '11px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', '#6d6d6d'),
  ('background-color', '#f7f7f9')
  ]

# Set CSS properties for td elements in dataframe
td_props = [
  ('font-size', '11px')
  ]

# Set table styles
styles = [
  dict(selector="th", props=th_props),
  dict(selector="td", props=td_props)
  ]

#.format({'B': "{:0<4.0f}", 'D': '{:+.2f}'})
rb2['Road Position']=rb2['Road Position'].astype(float)
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)
cm=sns.light_palette((210, 90, 60), input="husl",as_cmap=True)
#(df.style
s2=(rb2.style.applymap(color_negative, subset=[c for c in rb2.columns if isinstance(c, int)])
  .background_gradient(cmap=cm, subset=['Road Position' ])
  #.highlight_max(subset=['SS19 Overall'])
  .highlight_min(subset=['Overall Position'], color='lightgrey')
  .bar(subset=[c for c in rb2.columns if str(c).startswith('D')], align='zero', color=[ '#5fba7d','#d65f5f'])
  #.bar(subset=[c for c in rb2.columns if isinstance(c, int)], align='mid', color=[ '#5fba7d','#d65f5f'])
  .set_table_styles(styles)
)

  #.set_caption('This is a custom caption.')
  #.format({'total_amt_usd_pct_diff': "{:.2%}"})
s2

,Road Position,1,2,3,4,5,SS19 Overall,D1,D2,D3,D4,D5,D6,Overall Position,Overall Time
NEU,7,2.2,5.5,9.4,10.7,14.2,18.1,2.2,3.3,3.9,1.3,3.5,3.9,2,164.8
OGI,8,2.7,5.6,9.5,10.4,13.5,17.6,2.7,2.9,3.9,0.9,3.1,4.1,1,165.6
TÄN,9,2.2,5,8,8.2,11.1,14.1,2.2,2.8,3,0.2,2.9,3,8,-591.8
LAT,6,0.5,2.8,4.8,5.8,7.6,9.8,0.5,2.3,2.1,1,1.7,2.2,7,-511.7
SUN,10,0.8,0.7,1.3,1.8,2,2,0.8,-0.1,0.6,0.5,0.2,nan,11,-758.7
PAD,4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,4,nan
LAP,5,-0.7,-1.1,-1.2,-0.5,-0.4,-0.3,-0.7,-0.4,-0.1,0.7,0.1,0.1,3,53.5
BRE,2,nan,-1.2,-1,-1.6,-2,-3,nan,-1.1,0.1,-0.6,-0.4,-1,6,-98.8
OST,3,-0.6,-2.1,-3.2,-2.8,-3.6,-4.1,-0.6,-1.5,-1.2,0.4,-0.8,-0.5,5,-15.3
EVA,1,-4.1,-9.5,-14.2,-14.9,-18.7,-24,-4.1,-5.5,-4.6,-0.7,-3.9,-5.3,14,-907.4


In [42]:
rb2.style.bar()

,Road Position,1,2,3,4,5,SS19 Overall,D1,D2,D3,D4,D5,D6,Overall Position,Overall Time
NEU,7,2.2,5.5,9.4,10.7,14.2,18.1,2.2,3.3,3.9,1.3,3.5,3.9,2,164.8
OGI,8,2.7,5.6,9.5,10.4,13.5,17.6,2.7,2.9,3.9,0.9,3.1,4.1,1,165.6
TÄN,9,2.2,5,8,8.2,11.1,14.1,2.2,2.8,3,0.2,2.9,3,8,-591.8
LAT,6,0.5,2.8,4.8,5.8,7.6,9.8,0.5,2.3,2.1,1,1.7,2.2,7,-511.7
SUN,10,0.8,0.7,1.3,1.8,2,2,0.8,-0.1,0.6,0.5,0.2,nan,11,-758.7
PAD,4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,4,nan
LAP,5,-0.7,-1.1,-1.2,-0.5,-0.4,-0.3,-0.7,-0.4,-0.1,0.7,0.1,0.1,3,53.5
BRE,2,nan,-1.2,-1,-1.6,-2,-3,nan,-1.1,0.1,-0.6,-0.4,-1,6,-98.8
OST,3,-0.6,-2.1,-3.2,-2.8,-3.6,-4.1,-0.6,-1.5,-1.2,0.4,-0.8,-0.5,5,-15.3
EVA,1,-4.1,-9.5,-14.2,-14.9,-18.7,-24,-4.1,-5.5,-4.6,-0.7,-3.9,-5.3,14,-907.4


Problem with the bars is that the range is different in each column; ideally we want the same range in each column; could do this with two dummy rows to force max and min values?

In [43]:
from IPython.core.display import HTML 
HTML(s2.render().replace('nan',''))

,Road Position,1,2,3,4,5,SS19 Overall,D1,D2,D3,D4,D5,D6,Overall Position,Overall Time
NEU,7,2.2,5.5,9.4,10.7,14.2,18.1,2.2,3.3,3.9,1.3,3.5,3.9,2,164.8
OGI,8,2.7,5.6,9.5,10.4,13.5,17.6,2.7,2.9,3.9,0.9,3.1,4.1,1,165.6
TÄN,9,2.2,5,8,8.2,11.1,14.1,2.2,2.8,3,0.2,2.9,3,8,-591.8
LAT,6,0.5,2.8,4.8,5.8,7.6,9.8,0.5,2.3,2.1,1,1.7,2.2,7,-511.7
SUN,10,0.8,0.7,1.3,1.8,2,2,0.8,-0.1,0.6,0.5,0.2,,11,-758.7
PAD,4,,,,,,,,,,,,,4,
LAP,5,-0.7,-1.1,-1.2,-0.5,-0.4,-0.3,-0.7,-0.4,-0.1,0.7,0.1,0.1,3,53.5
BRE,2,,-1.2,-1,-1.6,-2,-3,,-1.1,0.1,-0.6,-0.4,-1,6,-98.8
OST,3,-0.6,-2.1,-3.2,-2.8,-3.6,-4.1,-0.6,-1.5,-1.2,0.4,-0.8,-0.5,5,-15.3
EVA,1,-4.1,-9.5,-14.2,-14.9,-18.7,-24,-4.1,-5.5,-4.6,-0.7,-3.9,-5.3,14,-907.4


In [53]:
import pandas as pd
import numpy as np

In [87]:
df=pd.DataFrame({'x1':list(np.random.randint(-10,10,size=10))+[-500,1000, -1000],
               'y1':list(np.random.randint(-5,5,size=13)),'y2':list(np.random.randint(-2,3,size=13)) })
df.style.bar( align='zero', color=[ '#5fba7d','#d65f5f'])

,x1,y1,y2
0,6,-1,2
1,-6,0,1
2,7,2,0
3,-10,-3,1
4,1,4,1
5,-1,-2,0
6,5,-5,0
7,4,-1,0
8,1,-1,-2
9,6,2,1


In [70]:
pandas.__version__

'0.23.0'

In [83]:
df['x2']= df['x'].clip(upper=10, lower=-10)
df.style.bar( align='zero', color=[ '#d65f5f','#5fba7d'])

,x,x2
0,-8,-8
1,-3,-3
2,9,9
3,9,9
4,6,6
5,-9,-9
6,4,4
7,-10,-10
8,-1,-1
9,-10,-10
